# Visualizing Molecular Orbitals

One of the most informative things that quantum mechanical calculations with molecules can give you is a visual picture of the molecular orbitals. The pictures can help explain patterns of reactivity. We can visualize these molecular orbitals right here in this notebook using the tools available to python.

## The Cube File

many QM programs can export "volumetric data" as cube files. this would be a data set of the value of the wavefunction at all points in space around a molecule. We can calculate the value at points in a xyz cubic lattice, hence the name. Then we can use math to visualize a surface that has a given value. Its just like contour lines on a map, but in 3-dimensions.

The value of the wavefunction chosen to map the surface is called the 'isovalue'. We generally choose a value so that bonding orbitals appear in scale with the van der Walls size of the molecule. A wavefunction can have positive or negative values and these are expressed as a pair of colors, often red and blue.

## The $\pi$ System of Butadiene

Let us use butadiene as an example and visualize the $\pi$ orbitals. First we must have a geometry with which to calculate the wavefunctions. I will build *s-trans* butadiene and enforce $C_{2h}$ symmetry in the calculation. Usinging the highest symmetry that applies to a geometry will make the best molecular orbitals. 

In [1]:
#### Build butadiene

# use psi4conda environment
import psi4
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import helpers as hp # Many useful functions from https://lcbc-epfl.github.io/iesm-public/intro.html

psi4.core.clean_options()

output_file = "butadiene_orb1.log"

psi4.set_memory("2GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

# The Z-matrix as a text string ### From previous optimization
data = """
       0 1
    C1
    C2      1    CC1
    C3      2    CC2           1  A_321
    C4      3    CC1           2  A_321           1     dihedral
    H5      1    CH_5_9        2  CH_512          3     D3
    H6      1    CH_6_10       5  CH_615          2     D2
    H7      2    CH_7_8        1  CH_721          3     D1
    H8      3    CH_7_8        4  CH_721          2     D1
    H9      4    CH_5_9        3  CH_512          2     D3
    H10     4    CH_6_10       9  CH_615          3     D2

       symmetry c2h
       dihedral  =  180     # s-trans
       CC1 = 1.3
       CC2 = 1.5
       CH_5_9 = 1.0
       CH_7_8 = 1.0
       CH_6_10 = 1.0
       A_321 = 120
       CH_512 = 120
       CH_615 = 120
       CH_721 = 120
       D1 = 180
       D2 = 180
       D3 = 180

       units angstrom
       """ 

# Create the Molecule object
mol = psi4.geometry(data)             # Create Molecule object from data string

# Optimize the structure
#psi4.set_options({"FROZEN_DIHEDRAL":"1 2 3 4"})

energy, wfn, hist = psi4.optimize("hf/6-31+G(d)", molecule = mol, return_wfn=True, return_history=True)
print("done")


  Memory set to   1.863 GiB by Python driver.
Optimizer: Optimization complete!
done


## Calculate the Wavefunctions

Most calculation driver functions in *Psi4* can calculate the wavefunctions if asked to do so. The ```return_wfn=True``` option will cause the function to return an additional object containing wavefunction data.

Below we calculate the energy of theoptimized geometry using a minimal basis set, STO-3G. We have a somewaht accurate structure (the hf/6-31+G(d) optimization only took a few seconds rather than a few weeks, so it's not as accurate as we could be; time is money.) Calculating the energy will not change the geometry but the results will be based on a smaller set of orbitals. I find this makes wavefunctions that are more similar to the diagrams in an organic text book. using larger absis sets mixes in empty d and f orbitals that are not important in interpreting the chemistry of second row elements like carbon. These extra orbitals will especially alter thenhigher energy antibonding orbitals. This effect is going to give greater accuracy but I am looking to acheive conceptual clarity, not extra decimal places.

In [2]:
energy, wfn = psi4.energy("hf/STO-3G", molecule = mol, return_wfn=True)

print("done")

#hp.drawXYZ(mol)

# psi4.set_output_file("temp.dat", append=False, print_header=False)
# mol.print_in_input_format()
# !cat "temp.dat"     # cat is a terminal command that will print the contents of a file to stdout

done


## Write the Cube Files

Below we set some options for the cube files and then use the ```psi4.cubeprop()``` function to calculate the cubuc wavefunction values for each orbital. As you can see in the code below, we can calculate just a few selected orbitals if we wish. 

In [5]:
psi4.set_options({
#        "CUBEPROP_ORBITALS": [11,12,-11,-12],      # default => []   ### positive orbital numbers for alpha electron, negative for beta.
         "CUBEPROP_ORBITALS": [],      # default => []
         "CUBEPROP_TASKS": "orbitals", # default => "orbitals";  frontier_orbitals, density, LOL, basis_functions, dual_description
         "CUBEPROP_FILEPATH": "./cube/"  
    })

psi4.cubeprop(wfn)

## Read in the Cube Files and Visualize Orbitals

The code below will call a function that will read in all the cube files in a directory and present the molecular orbitals visually. By simple counding of electron paire I know that the HOMO will be the 15th orbital (ranked in energy from loswest to highest). I can move the slider to identify the other $\pi$ orbitals.

In [7]:
hp.show_orbitals(wd="./cube/", mol=mol)

interactive(children=(Dropdown(description='Orbital:', options=('1', '10', '11', '12', '13', '14', '15', '16',…